In [16]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import sys
import os
from pathlib import Path

os.chdir(path="/Research")
print(Path.cwd())
sys.path.append(os.path.abspath('/Research/gpudrive'))

from gpudrive.env.config import EnvConfig, SceneConfig
from gpudrive.env.env_torch import GPUDriveTorchEnv
from gpudrive.visualize.utils import img_from_fig
from gpudrive.env.dataset import SceneDataLoader

/Research


In [17]:
import importlib

from CMTR.mtr.config import cfg_from_list
# from CMTR.mtr.datasets.dataset import DatasetTemplate
from CMTR.mtr.datasets.waymo.waymo_dataset import WaymoDataset
# from CMTR.mtr.models.context_encoder import MTREncoder
import yaml 
from easydict import EasyDict
import logging
import torch
import numpy as np

# import CMTR.mtr.datasets.waymo.waymo_dataset
# importlib.reload(CMTR.mtr.datasets.waymo.waymo_dataset)
# from CMTR.mtr.datasets.waymo.waymo_dataset import WaymoDataset



In [18]:
device = "cuda:0"
MAX_NUM_OBJECTS = 32  # Maximum number of objects in the scene we control
NUM_WORLDS = 2  # Number of parallel environments
K_UNIQUE_SCENES = 1 # Number of unique scenes

In [19]:
def getconfig(cfg_file):
    with open(cfg_file, 'r') as f:
        try:
            config = yaml.load(f, Loader=yaml.FullLoader)
        except:
            config = yaml.load(f)

    return EasyDict(config)

logger = logging.getLogger(__name__)

config = getconfig("dev/configs/config.yaml")
waymo_dataset = WaymoDataset(config.DATA_CONFIG, test_mode=1, logger=logger)

In [20]:

DYNAMICS_MODEL = "delta_local" # "delta_local" / "state" / "classic"
DATA_PATH = "data/processed/examples" # Your data path
MAX_NUM_OBJECTS = 64
NUM_ENVS = 2
DEVICE = "cuda:0"

# Configs
env_config = EnvConfig(dynamics_model=DYNAMICS_MODEL)

In [6]:
# Make dataloader
data_loader = SceneDataLoader(
    root="gpudrive/data/processed/examples", # Path to the dataset
    batch_size=NUM_ENVS, # Batch size, you want this to be equal to the number of worlds (envs) so that every world receives a different scene
    dataset_size=NUM_ENVS, # Total number of different scenes we want to use
    sample_with_replacement=False, 
    seed=42, 
    shuffle=True,   
)

print("Making the environment")

# Make environment
env = GPUDriveTorchEnv(
    config=env_config,
    data_loader=data_loader,
    max_cont_agents=MAX_NUM_OBJECTS, # Maximum number of agents to control per scenario
    device="cuda", 
    action_type="continuous" # "continuous" or "discrete"
)

Making the environment
Compiling GPU engine code:
/Research/gpudrive/external/madrona/src/mw/device/memory.cpp
/Research/gpudrive/external/madrona/src/mw/device/state.cpp
/Research/gpudrive/external/madrona/src/mw/device/crash.cpp
/Research/gpudrive/external/madrona/src/mw/device/consts.cpp
/Research/gpudrive/external/madrona/src/mw/device/taskgraph.cpp
/Research/gpudrive/external/madrona/src/mw/device/taskgraph_utils.cpp
/Research/gpudrive/external/madrona/src/mw/device/sort_archetype.cpp
/Research/gpudrive/external/madrona/src/mw/device/host_print.cpp
/Research/gpudrive/external/madrona/src/mw/../common/hashmap.cpp
/Research/gpudrive/external/madrona/src/mw/../common/navmesh.cpp
/Research/gpudrive/external/madrona/src/mw/../core/base.cpp
/Research/gpudrive/external/madrona/src/mw/../physics/physics.cpp
/Research/gpudrive/external/madrona/src/mw/../physics/geo.cpp
/Research/gpudrive/external/madrona/src/mw/../physics/xpbd.cpp
/Research/gpudrive/external/madrona/src/mw/../physics/tgs.c

/Research/gpudrive/src/level_gen.cpp(283): warning #177-D: function "madrona_gpudrive::createFloorPlane" was declared but never referenced
  static void createFloorPlane(Engine &ctx)
              ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"





In [21]:
from gpudrive.datatypes.roadgraph import GlobalRoadGraphPoints
from gpudrive.datatypes.observation import (
    LocalEgoState,
    GlobalEgoState,
    PartnerObs,
)

In [22]:

device = "cuda:0"

def getconfig(cfg_file):
    with open(cfg_file, 'r') as f:
        try:
            config = yaml.load(f, Loader=yaml.FullLoader)
        except:
            config = yaml.load(f)

    return EasyDict(config)

logger = logging.getLogger(__name__)

config = getconfig("dev/configs/config.yaml")
waymo_dataset = WaymoDataset(config.DATA_CONFIG, test_mode=1, logger=logger)


In [23]:
import pickle

In [24]:
index = 0
info = waymo_dataset.infos[index]
scene_id = info['scenario_id']
with open(waymo_dataset.data_path / f'sample_{scene_id}.pkl', 'rb') as f:
    info = pickle.load(f)


In [25]:
for i in info: 
    print(i)

track_infos
dynamic_map_infos
map_infos
scenario_id
timestamps_seconds
current_time_index
sdc_track_index
objects_of_interest
tracks_to_predict


In [26]:
info['track_infos']['trajs'].shape

(33, 91, 10)

In [27]:
global_ego = GlobalEgoState.from_tensor(
    abs_self_obs_tensor=env.sim.absolute_self_observation_tensor(),
    backend=env.backend,
    device=device
)

global_ego.pos_x

tensor([[ -6.7849,  11.9054,  -2.4020, -21.0901,   7.9561, -23.5463, -36.5822,
         -41.3113, -12.2128, -11.7589,   3.9974, -47.6550,  13.9465, -16.8408,
         -14.7782,  38.1151, -48.5161, -42.1628, -25.2578,  14.7762,  19.9061,
          39.3571,  44.5558,  20.5291,  28.0392, -24.7926, -53.1283, -23.1023,
          -9.6686, -23.1460, -29.2159, -32.9471,  -5.5679, -33.6177, -49.9692,
          17.5591,  25.8573, -38.3859, -61.9923,  19.5956, -42.4196,   7.7034,
         -12.6141,  55.4182, -35.0979, -20.7358,  -7.4191,   0.0000,   0.0000,
           0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000],
        [ 40.6240,  22.0947,  67.0645,  29.4111,  33.4258,  58.2207,  56.1846,
         -27.5264,  41.8486,  65.6924,  -9.1709,  83.8340,  44.9922,  -8.8115,
         -27.9912,  82.6572, -23.9619,  22.2432, -11.4463, -16.9287,  74.8994,
          -3.2803, -13.4932,  65

In [28]:
info_tensor = env.sim.info_tensor() 

In [29]:
import importlib
import imitation_data_generation
importlib.reload(imitation_data_generation)
from imitation_data_generation import generate_state_action_pairs

ModuleNotFoundError: No module named 'imitation_data_generation'

In [30]:
obs, act, _, _ = generate_state_action_pairs(env=env,
                            device="cuda",
                            action_space_type="continuous", 
                            use_action_indices=True,  
                            make_video=False,
                            render_index=[2, 0],
                            save_path="use_discr_actions_fix",
                            CMTR=True, 
                            waymo_dataset=waymo_dataset,
                            prior_frame=10,
                            future_frame=5
) 

NameError: name 'generate_state_action_pairs' is not defined

In [159]:
import waymo_iterator
importlib.reload(waymo_iterator)
from waymo_iterator import TrajectoryIterator
from torch.utils.data import DataLoader

In [160]:
traj_iterator = TrajectoryIterator(
    env=env,
    scene_data_loader=data_loader,
    cmtr=True,
    waymo_dataset=waymo_dataset,
    prior_frame=5,
    future_frame=5
)

dataset = iter(
        DataLoader(
            traj_iterator,
            batch_size=2,
            collate_fn=traj_iterator.collate_batch,
    )
)

In [161]:
data = next(dataset)

In [162]:
data

({'batch_size': 2,
  'index': array([None, None], dtype=object),
  'input_dict': {'scenario_id': array(['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
          '0', '0', '0'], dtype='<U1'),
   'obj_trajs': tensor([[[[-2.5419e+01,  6.1377e+00,  0.0000e+00,  ...,  0.0000e+00,
               0.0000e+00,  0.0000e+00],
             [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
               0.0000e+00,  0.0000e+00],
             [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
               0.0000e+00,  0.0000e+00],
             [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
               0.0000e+00,  0.0000e+00],
             [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
               0.0000e+00,  0.0000e+00]],
   
            [[-8.3888e-01,  3.6648e-03,  0.0000e+00,  ...,  1.4529e-02,
              -2.3994e+00,  5.9760e-03],
             [-6.3134e-01,  4.2064e-03,  0.0000e+00,  ...,  1.5126e-02,
              -2.3994e+